# 数据降维与可视化

目的是查看数据形状，选择合适的聚类算法

In [ ]:
from sqlalchemy import create_engine
from config import MySQLConfig
import pandas as pd
from matplotlib import rcParams
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
# 设置中文显示
rcParams['font.sans-serif'] = ['SimHei']
rcParams['axes.unicode_minus'] = False

In [ ]:
db = MySQLConfig()
engine_str = (
    f"mysql+pymysql://{db.user}:{db.password}"
    f"@{db.host}:{db.port}/{db.database}?charset={db.charset}"
)
engine = create_engine(engine_str)

print("MySQL连接成功")

query = "SELECT * FROM DeliverInsight.customer"
df = pd.read_sql(query, engine)

print("数据加载完成")
df.head()

## 1. 主成分分析（PCA, Principal Component Analysis）


##### （1）定义

PCA 是一种**线性降维方法**，通过正交变换将高维数据映射到低维空间，使新坐标系的各维度（主成分）依次最大化数据方差，从而保留数据中尽可能多的信息。

##### （2）输入

* 数据集
  $$
  X = {X_1, X_2, \dots, X_n}, \quad X_i \in \mathbb{R}^d
  $$

##### （3）步骤与公式

1. **中心化数据**
   $$
   \tilde{X} = X - \bar{X}, \quad \bar{X} = \frac{1}{n} \sum_{i=1}^{n} X_i
   $$

2. **计算协方差矩阵**
   $$
   \Sigma = \frac{1}{n} \tilde{X}^\top \tilde{X} \in \mathbb{R}^{d \times d}
   $$

3. **特征分解**
   $$
   \Sigma \mathbf{v}_k = \lambda_k \mathbf{v}_k, \quad k=1,2,\dots,d
   $$
   其中 $\lambda_k$ 为特征值，$\mathbf{v}_k$ 为对应特征向量。

4. **降维映射**
   选择前 $m$ 个最大特征值对应的特征向量构成 $\mathbf{V}_m = [\mathbf{v}_1, \dots, \mathbf{v}_m]$，映射到低维空间：
   $$
   Y = \tilde{X} \mathbf{V}_m \in \mathbb{R}^{n \times m}
   $$

> 核心思想：寻找数据方差最大的方向，保留最重要信息。


### PCA 2D 降维可视化

In [ ]:
# ===============================
# 1. 选择数值型字段并标准化
# ===============================
num_df = df.select_dtypes(include=['float64', 'int64']).dropna()
scaler = StandardScaler()
scaled = scaler.fit_transform(num_df)

# ===============================
# 2. PCA 降维
# ===============================
pca = PCA(n_components=2)
pca_data = pca.fit_transform(scaled)

# ===============================
# 3. 输出 PCA 结果
# ===============================
pca_df = pd.DataFrame(
    pca_data,
    columns=["PC1", "PC2"]
)
display(pca_df.head(5))  # 查看前 5 行

print("前两个主成分累计解释方差：", pca.explained_variance_ratio_.sum())

# ===============================
# 4. 查看每个字段对主成分的贡献（权重 / 系数）
# ===============================
loadings = pd.DataFrame(
    pca.components_.T,
    columns=["PC1", "PC2"],
    index=num_df.columns
).round(3)

display(loadings)

# 可视化贡献大小（条形图）
loadings.plot(kind="bar", figsize=(12, 6))
plt.title("每个字段对主成分的贡献")
plt.ylabel("贡献系数")
plt.grid(True)
plt.show()

# ===============================
# 5. PCA 可视化
# ===============================
# 散点透明化
plt.figure(figsize=(10, 7))
plt.scatter(
    pca_data[:, 0],
    pca_data[:, 1],
    s=8,
    alpha=0.15,
    edgecolors='none'
)
plt.title("PCA 2D 降维（Scatter）")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.grid(True)
plt.show()

# Hexbin 密度图
plt.figure(figsize=(10, 7))
plt.hexbin(
    pca_data[:, 0],
    pca_data[:, 1],
    gridsize=40,
    cmap='viridis'
)
plt.colorbar(label="密度")
plt.title("PCA 2D 降维（Hexbin 密度图）")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

# Joint KDE 分布图
plt.figure(figsize=(8, 8))
sns.jointplot(
    x=pca_data[:, 0],
    y=pca_data[:, 1],
    kind="kde",
    fill=True,
    cmap="Blues",
    height=8
)
plt.suptitle("PCA 2D 分布（Joint KDE）", y=1.02)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()


### PCA 3D 降维可视化

In [ ]:
# ===============================
# 1. 选择数值型字段并标准化
# ===============================
num_df = df.select_dtypes(include=['float64', 'int64']).dropna()
scaler = StandardScaler()
scaled = scaler.fit_transform(num_df)

# ===============================
# 2. PCA 降维到 3 维
# ===============================
pca = PCA(n_components=3)
pca_data = pca.fit_transform(scaled)

# ===============================
# 3. 输出 PCA 结果
# ===============================
pca_df = pd.DataFrame(
    pca_data,
    columns=["PC1", "PC2", "PC3"]
)
display(pca_df.head(5))  # 查看前 5 行
print("前三个主成分累计解释方差：", pca.explained_variance_ratio_.sum())

# ===============================
# 4. 查看每个字段对主成分的贡献（权重 / 系数）
# ===============================
loadings = pd.DataFrame(
    pca.components_.T,
    columns=["PC1", "PC2", "PC3"],
    index=num_df.columns
).round(3)

display(loadings)

# 可视化贡献大小（条形图）
loadings.plot(kind="bar", figsize=(12, 6))
plt.title("每个字段对主成分的贡献")
plt.ylabel("贡献系数")
plt.grid(True)
plt.show()

# ===============================
# 5. PCA 3D 可视化
# ===============================
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(
    pca_data[:, 0],
    pca_data[:, 1],
    pca_data[:, 2],
    s=20,
    alpha=0.2,
    edgecolors='none'
)
ax.set_title("PCA 3D 降维散点图")
ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
ax.set_zlabel("PC3")
plt.show()

## 2. t-分布随机邻域嵌入（t-SNE, t-distributed Stochastic Neighbor Embedding）


##### （1）定义

t-SNE 是一种**非线性降维方法**，将高维数据映射到低维空间（2D 或 3D），主要保持局部邻居结构，使相似样本在低维空间中靠近。

##### （2）输入

* 数据集
  $$
  X = {X_1, X_2, \dots, X_n}, \quad X_i \in \mathbb{R}^d
  $$

##### （3）步骤与公式

1. **高维空间相似度**
   条件概率：
   $$
   p_{j|i} = \frac{\exp\left(-|x_i - x_j|^2 / 2\sigma_i^2 \right)}{\sum_{k \neq i} \exp\left(-|x_i - x_k|^2 / 2\sigma_i^2 \right)}
   $$
   对称化：
   $$
   p_{ij} = \frac{p_{j|i} + p_{i|j}}{2n}
   $$

2. **低维空间相似度**
   使用 t 分布：
   $$
   q_{ij} = \frac{(1 + |y_i - y_j|^2)^{-1}}{\sum_{k \neq l} (1 + |y_k - y_l|^2)^{-1}}
   $$

3. **最小化 KL 散度**
   $$
   C = KL(P | Q) = \sum_{i \neq j} p_{ij} \log \frac{p_{ij}}{q_{ij}}
   $$
   通过梯度下降优化低维坐标。

> 核心思想：保持局部邻居相似关系，允许远距离关系扭曲。



### t-SNE 2D 降维可视化

In [ ]:
tsne = TSNE(n_components=2, learning_rate='auto', init='pca', random_state=42)
tsne_data = tsne.fit_transform(scaled)
df_tsne = pd.DataFrame(tsne_data, columns=['t-SNE1', 't-SNE2'])
#  散点透明化
plt.figure(figsize=(10, 7))
plt.scatter(
    tsne_data[:, 0],
    tsne_data[:, 1],
    s=10,
    alpha=0.2,
    c='steelblue',
    edgecolors='none'
)
plt.title("t-SNE 2D 降维（散点透明图）")
plt.xlabel("t-SNE1")
plt.ylabel("t-SNE2")
plt.grid(True)
plt.show()

#  Hexbin 密度图
plt.figure(figsize=(10, 7))
plt.hexbin(
    tsne_data[:, 0],
    tsne_data[:, 1],
    gridsize=50,
    cmap='viridis'
)
plt.colorbar(label="密度")
plt.title("t-SNE 2D 降维（Hexbin 密度图）")
plt.xlabel("t-SNE1")
plt.ylabel("t-SNE2")
plt.show()


### t-SNE 3D 降维可视化

In [ ]:
tsne = TSNE(n_components=3, learning_rate='auto', init='pca', random_state=42)
tsne_data = tsne.fit_transform(scaled)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(
    tsne_data[:, 0],
    tsne_data[:, 1],
    tsne_data[:, 2],
    s=15,
    alpha=0.2,
    c='steelblue',
    edgecolors='none'
)
ax.set_title("t-SNE 3D 降维散点图")
ax.set_xlabel("t-SNE1")
ax.set_ylabel("t-SNE2")
ax.set_zlabel("t-SNE3")
plt.show()


## PCA 与 t-SNE 对比

| 特性        | PCA           | t-SNE                  |
| --------- | ------------- | ---------------------- |
| **类型**    | 线性降维          | 非线性降维                  |
| **目标**    | 保留方差，尽量保留全局信息 | 保留局部结构，反映邻近关系          |
| **数学基础**  | 协方差矩阵特征分解     | 高维/低维概率分布匹配 (KL 散度最小化) |
| **适合数据**  | 线性或近似线性       | 高维非线性复杂分布              |
| **输出解释性** | 可解释主成分方向      | 降维后坐标难直接解释             |
| **计算复杂度** | O(d³)         | O(n²)，大数据较慢            |
| **常用用途**  | 数据压缩、训练加速     | 可视化、聚类探索、异常检测          |## t-SNE